In [2]:
import sys
sys.path.insert(0, '/Users/zacksiegfried/Documents/methylspan')
import MSM
import MethylMapping


all_files = MSM.pullMethylMetaData('pancreas')

# selecting only methylation assay 450 files
all_files2 = []
for i in all_files:
    if i['platform'] == 'Illumina Human Methylation 450':
        all_files2.append(i)

# not gaureteed to return same number of cases, (remove file_id from sample data if missing from meta data)
# missing values dropped anyways in next steps
sample_data = MSM.methylDataFormat(all_files2, 50)
meta_data = MSM.metaDataFormat(sample_data, all_files2)

map_test = MethylMapping.methylMap450(sample_data)

Number of methylation files for all cases for primary site pancreas: 471
Final number of cases in methylation dataset : 50
Final number of cases in metadata dataset : 50


In [3]:
#map_test.to_csv('data/map_test.csv')

from collections import Counter
gene_dict = Counter(map_test.columns)
#print(gene_dict)

In [39]:
### AVERAGE GENE REGION METHOD
import MethylMapping

sample_average_data = MethylMapping.averageGene(map_test, 'MGMT')

                                          MGMT
id                                            
7ac9d292-eac7-409f-b0e8-9c1a960d7401  0.815752
37a63eb3-049c-47e3-84ff-7de36074e5cd  0.813167
976cd416-5ce3-4ceb-a6a8-00d565b7cf48  0.822057
1e61d269-6169-4ed5-b0dc-d3dbf5dadc37  0.812727
b102cef3-f320-43bd-9bcd-052ab9bc105c  0.817987


In [17]:
### FULL GENE REGION METHOD
import MethylMapping

sample_full_data = MethylMapping.fullGene(map_test, 'MGMT')

                                        MGMT_1    MGMT_2    MGMT_3    MGMT_4  \
7ac9d292-eac7-409f-b0e8-9c1a960d7401  0.013851  0.017323  0.960409  0.947207   
37a63eb3-049c-47e3-84ff-7de36074e5cd  0.013455  0.017649  0.960832  0.942600   
976cd416-5ce3-4ceb-a6a8-00d565b7cf48  0.012386  0.014826  0.970702  0.972389   
1e61d269-6169-4ed5-b0dc-d3dbf5dadc37  0.011619  0.016112  0.956667  0.962331   
b102cef3-f320-43bd-9bcd-052ab9bc105c  0.024979  0.102605  0.962408  0.957829   

                                        MGMT_5    MGMT_6    MGMT_7    MGMT_8  \
7ac9d292-eac7-409f-b0e8-9c1a960d7401  0.891783  0.028081  0.017177  0.026706   
37a63eb3-049c-47e3-84ff-7de36074e5cd  0.887554  0.293068  0.086225  0.030847   
976cd416-5ce3-4ceb-a6a8-00d565b7cf48  0.942547  0.152831  0.121371  0.074536   
1e61d269-6169-4ed5-b0dc-d3dbf5dadc37  0.910228  0.033816  0.019733  0.027769   
b102cef3-f320-43bd-9bcd-052ab9bc105c  0.897508  0.229302  0.175651  0.136434   

                                      

In [40]:
### FORMATTING FOR SURVIVAL

import pandas as pd
from lifelines import CoxPHFitter
from lifelines.utils import k_fold_cross_validation

def mappingSurvivalFormat(sample_data, meta_data):
    
    meta_data.index.name = 'id'
    sample_data.index.name = 'id'
    sample_data = sample_data.dropna(axis=1)

    # merges dfs and formats 
    full_data = pd.merge(meta_data, sample_data, on = 'id')
    full_data.replace(['Dead', 'Alive'],[1, 0], inplace=True)
    full_data = full_data[full_data['days_to_death'].notna()]

    print("Final number of cases used: " + str(full_data.shape[0]))

    return(full_data)


full_data = mappingSurvivalFormat(sample_average_data, meta_data)

Final number of cases used: 50


In [43]:
### SURVIVAL ANALYSIS EXECUTION & TESTING

# model trained on complete data set 
cph = CoxPHFitter()
cph.fit(full_data, duration_col='days_to_death', event_col='vital_status')
print(cph.confidence_intervals_)

# Concordance index (will remodel cph.)
scores = k_fold_cross_validation(cph, full_data, 'days_to_death', event_col='vital_status', k=5, scoring_method="concordance_index")
print(scores)

           95% lower-bound  95% upper-bound
covariate                                  
MGMT            -62.422887         7.697958
[0.6129032258064516, 0.4444444444444444, 0.7096774193548387, 0.6571428571428571, 0.6521739130434783]


/Users/zacksiegfried/opt/anaconda3/lib/python3.9/site-packages/lifelines/utils/__init__.py:1102: ConvergenceWarning: Column(s) ['MGMT'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
